Description
--

We need a Google Cloud Platform project with billing enabled.</br>
We first have to enable the `Cloud AI Platform Models API` and the `Compute Engine API`.</br>
We then create an AI Platform Notebooks instance, then we select the latest `PyTorch` instance type (without/with ?????To determine?????? GPUs).</br>
Once the instance has been created, we select `Open JupyterLab`:</br></br>
After setting up the environment, we run the folling commands :

In [1]:
!gsutil version -l

gsutil version: 5.6
checksum: 0988591864f8c3a8ca1ae43f519e6d15 (OK)
boto version: 2.49.0
python version: 3.8.11 (default, Dec 23 2021, 04:09:42) [Clang 12.0.1 ]
OS: Linux 4.19.0-18-cloud-amd64
multiprocessing available: True
using cloud sdk: True
pass cloud sdk credentials to gsutil: True
config path(s): /etc/boto.cfg
gsutil path: /usr/lib/google-cloud-sdk/bin/gsutil
compiled crcmod: True
installed via package manager: False
editable install: False


In [26]:
GCP_PROJECT = 'My First Project'
BUCKET_URL = 'gs://storage_bucket_speech'

In [3]:
!gsutil mb $BUCKET_URL

Creating gs://storage_bucket_speech/...
ServiceException: 409 A Cloud Storage bucket named 'storage_bucket_speech' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


The bucket already exists because it was already created

 Store the data on Google Storage
 ---

In [32]:
from IPython.display import clear_output

In [25]:
train_path = '../feature-extraction/train/'
test_path = '../feature-extraction/test/'
valid_path = '../feature-extraction/valid/'

In [10]:
!gsutil -m cp -r $train_path $BUCKET_URL
!gsutil -m cp -r $test_path $BUCKET_URL
!gsutil -m cp -r $valid_path $BUCKET_URL
clear_output()

In [21]:
from trainer.config import Config as cfg

In [23]:
cfg.N_MELS

128

In [19]:
!touch ./trainer/model.py

In [1]:
from google.cloud import storage

In [2]:
import os, logging, json, pickle, argparse

In [3]:
!pip install fastavro

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 33.5 MB/s eta 0:00:00a 0:00:01


In [4]:
!pip install avro

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.4/83.4 KB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for avro: filename=avro-1.11.0-py2.py3-none-any.whl size=115926 sha256=a4198cdf24d1ae6ba8f99c3f65cd8230c458c9653a6393aa770e19b49ea56d49
  Stored in directory: /home/jupyter/.cache/pip/wheels/7d/79/ec/d7acfd56e9934b311783689c07ffecf6af9bde172950927f6d
Successfully built avro


In [16]:
import avro.schema
from avro.datafile import DataFileReader, DataFileWriter
from avro.io import DatumReader, DatumWriter

In [10]:
import json

In [20]:
schema = {"namespace": "example.avro",
 "type": "record",
 "name": "User",
 "fields": [
     {"name": "name", "type": "string"},
     {"name": "favorite_number",  "type": ["int", "null"]},
     {"name": "favorite_color", "type": ["string", "null"]}
 ]
}

In [21]:
schema = avro.schema.parse(json.dumps(schema))

In [22]:
writer = DataFileWriter(open("users.avro", "wb"), DatumWriter(), schema)
writer.append({"name": "Alyssa", "favorite_number": 256})
writer.append({"name": "Ben", "favorite_number": 7, "favorite_color": "red"})
writer.close()

In [23]:
reader = DataFileReader(open("users.avro", "rb"), DatumReader())
for user in reader:
    print(user)
reader.close()

{'name': 'Alyssa', 'favorite_number': 256, 'favorite_color': None}
{'name': 'Ben', 'favorite_number': 7, 'favorite_color': 'red'}


In [27]:
import os

In [31]:
os.path.join(BUCKET_URL,'test')

'gs://storage_bucket_speech/test'

In [49]:
import subprocess

In [47]:
TRAIN_DATA_DIR = 'tr'
LOCAL_PATH = 'loc'

if not os.path.exists(TRAIN_DATA_DIR):
    os.mkdir(LOCAL_PATH)
    
if not os.path.exists(TRAIN_DATA_DIR):
    os.mkdir(os.path.join(LOCAL_PATH, TRAIN_DATA_DIR))

In [39]:
!mkdir test_dir

In [42]:
len(os.listdir(download_path))

303

In [40]:
download_path = './test_dir'
gcs_dir_path = 'gs://storage_bucket_speech/test/*.pickle'

In [43]:
!gsutil -q -m cp $gcs_dir_path $download_path 

In [44]:
class rr:
    
    BUCKET_URL = 'gs://storage_bucket_speech'
    TRAIN_DIR  = ''
    VALID_DIR  = ''
    TEST_DIR   = ''
    
    #Model parameters :
    N_MELS              = 128
    INPUT_SPEC_SIZE     = 3 * N_MELS
    RNN_CELL            = 'lstm' # 'lstm' | 'gru'
    CNN_FILTER_SIZE     = 64
    
    NUM_GENDER_CLASSES  = 2
    NUM_EMOTION_CLASSES = 8

In [46]:
rr.CNN_FILTER_SIZE

64

In [2]:
import torch

In [3]:
torch.cuda.is_available()

False

In [4]:
torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device(type='cpu')

In [8]:
(torch.tensor([1]), torch.tensor([1])).to(device)

AttributeError: 'tuple' object has no attribute 'to'